In [1]:
import ollama
import pandas as pd
import json

from pdfminer.high_level import extract_text
import re
import pandas as pd
from nameparser import HumanName
from tqdm import tqdm

In [ ]:



pdf_path = "../data/ERC-2024-AdG-panel-members.pdf"

pdf_text = extract_text(pdf_path)



# Schritt 2: Prompt für Ollama erstellen
prompt = f"""Extrahiere alle Personen-Informationen aus dem folgenden Text. 
Gib die Daten als eine JSON-Liste von Objekten zurück. Jedes Objekt sollte die Schlüssel 'Name', 'Domain', 'Subdomain', 'Typ' (Chair/Member), 'ERC-Datum' enthalten. 


Text:
{pdf_text}

Ausgabe:"""

# Schritt 3: Ollama aufrufen
response = ollama.chat(
    model='mistral',
    messages=[{'role': 'user', 'content': prompt}]
)




ResponseError: model 'mistral' not found (status code: 404)

In [ ]:
# JSON-Antwort verarbeiten und DataFrame erstellen
try:
    json_response = response['message']['content']
    data = json.loads(json_response)
    df = pd.DataFrame(data)
    print(df)
except (json.JSONDecodeError, KeyError) as e:
    print("Fehler beim Verarbeiten der LLM-Antwort:", e)